# Assignment: Segmenting and Clustering Neighborhoods in Toronto

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import folium # map rendering library
import requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans # import k-means from clustering stage
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## Part 1: Data Wrangling
I used Beautiful Soup based on this cool tutorial by Scott Rome: https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/  
Read postcode data for Toronto from Wiki page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### First define my modified version of the parse table method by Scott Rome

In [2]:
def parse_html_table(table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text().strip('\n'))

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text().strip(')\n')  # Get rid of the \n 
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df

### Extract the table
Read the table and parse it using the method above

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
table = soup.find("table")

In [4]:
toronto_pc_df = parse_html_table(table)
toronto_pc_df['Postal code'].str.strip() # Make sure there are no leaked spaces in the postal code
toronto_pc_df.shape

(180, 3)

In [5]:
toronto_pc_df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Drop the Not assigned boroughs.

In [6]:
toronto_pc_df.drop(toronto_pc_df[toronto_pc_df['Borough'].str.contains('Not assigned')].index, inplace=True)
toronto_pc_df.sort_values(by=['Postal code'], inplace=True)
toronto_pc_df.reset_index(drop=True, inplace=True)

In [7]:
toronto_pc_df

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


### Part 1 Results

In [8]:
print('Toronto has {} unique postal codes in use as of 30 March 2020.'.format(len(toronto_pc_df['Postal code'].unique())))

Toronto has 103 unique postal codes in use as of 30 March 2020.


In [9]:
toronto_pc_df.shape

(103, 3)

## Part 2: Location Data
Get the latitude and the longitude coordinates of each neighborhood.
I'm using the data provided in a CSV file as geocoder is not working

### Read the data into a dataframe

In [10]:
toronto_geo_df = pd.read_csv('Geospatial_Coordinates.csv')

### Find the location data the postcodes in use

In [11]:
toronto_geo_df = toronto_geo_df[toronto_geo_df['Postal Code'].isin(toronto_pc_df['Postal code'])]
toronto_geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the two dataframes

In [12]:
toronto_df = toronto_pc_df
toronto_df['Latitude'] = toronto_geo_df['Latitude']
toronto_df['Longitude'] = toronto_geo_df['Longitude']

### Part 2 Results

In [13]:
toronto_df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.688905,-79.554724
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437


## Part 3: Clustering Map Visualisation

### Get the Toronto location

In [14]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for pc, lat, lng, borough, neighborhood in zip(toronto_df['Postal code'], toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}, {}'.format(pc, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### From here on I'm going to repeat the clustering technique used in the workshop for NY on the Toronto data.

#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = '543RCQIDW44OE0ZDAPA5H5I3WDPAQTJZBSA1IIRNMYOV2B4W' # your Foursquare ID
CLIENT_SECRET = 'HTU223QZD4GOZ3W2H424YP2CZ1LGKOF1KSYNKICNEOYAV1V2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 543RCQIDW44OE0ZDAPA5H5I3WDPAQTJZBSA1IIRNMYOV2B4W
CLIENT_SECRET:HTU223QZD4GOZ3W2H424YP2CZ1LGKOF1KSYNKICNEOYAV1V2


#### Define function to explore neighborhoods

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [18]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                 latitudes=toronto_df['Latitude'],
                                 longitudes=toronto_df['Longitude'])

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

#### Let's check the size of the resulting dataframe

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(2197, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


Let's check how many venues were returned for each neighborhood

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Alderwood / Long Branch,8,8,8,8,8,8
Bathurst Manor / Wilson Heights / Downsview North,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,24,24,24,24,24,24
...,...,...,...,...,...,...
Willowdale,40,40,40,40,40,40
Woburn,3,3,3,3,3,3
Woodbine Heights,9,9,9,9,9,9


#### Let's find out how many unique categories can be curated from all the returned venues

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


#### Now let's analyse all the neighbourhoods

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [23]:
toronto_onehot.shape

(2197, 270)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,Alderwood / Long Branch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,Bedford Park / Lawrence Manor East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.025,0.0,0.0,0.0,0.0,0.0,0.0
89,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
90,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.111111,0.000,0.0,0.0,0.0,0.0,0.0,0.0
91,York Mills / Silver Hills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0


#### Let's confirm the new size

In [25]:
toronto_grouped.shape

(93, 270)

#### Let's print each neighborhood along with the top 5 most common venues

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1                     Lounge   0.2
2             Clothing Store   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Alderwood / Long Branch----
            venue  freq
0     Pizza Place  0.25
1        Pharmacy  0.12
2             Gym  0.12
3  Sandwich Place  0.12
4    Skating Rink  0.12


----Bathurst Manor / Wilson Heights / Downsview North----
                venue  freq
0                Bank  0.10
1         Coffee Shop  0.10
2  Chinese Restaurant  0.05
3          Restaurant  0.05
4         Gas Station  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4        Moving Target  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1      Sandwich Place  0.08
2          Restaurant  0.08
3  Italia

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Discount Store
1,Alderwood / Long Branch,Pizza Place,Pharmacy,Gym,Skating Rink,Coffee Shop,Pub,Sandwich Place,Dog Run,Diner,Discount Store
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Gas Station,Fried Chicken Joint,Restaurant,Shopping Mall,Middle Eastern Restaurant,Deli / Bodega,Pizza Place,Supermarket
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Bedford Park / Lawrence Manor East,Italian Restaurant,Sandwich Place,Coffee Shop,Restaurant,Pharmacy,Butcher,Thai Restaurant,Juice Bar,Fast Food Restaurant,Café


<a id='item4'></a>

#### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [29]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
unique, counts = np.unique(kmeans.labels_, return_counts=True)
dict(zip(unique, counts))

{0: 6, 1: 26, 2: 1, 3: 48, 4: 10, 5: 1, 6: 1}

I get the best distribution of neighbourhoods when using 7 clusters.

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,1.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,3.0,Bar,History Museum,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,1.0,Moving Target,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Bank,Intersection,Medical Center,Mexican Restaurant,Yoga Studio
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Caribbean Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Bank,Hakka Restaurant,Electronics Store,Eastern European Restaurant


Let's check whether all neighbourhoods have been properly clustered.

In [31]:
print('There are {} neighbourhoods without results.'.format(toronto_merged['Cluster Labels'].isnull().sum()))

There are 5 neighbourhoods without results.


Let's make sure we drop those neighbourhoods.

In [32]:
toronto_merged.dropna(axis=0, inplace=True)

Finally, let's visualize the resulting clusters

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,M6B,Glencairn,0.0,Park,Pizza Place,Japanese Restaurant,Pub,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
81,M6N,Runnymede / The Junction North,0.0,Pizza Place,Grocery Store,Convenience Store,Bus Line,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
89,M8W,Alderwood / Long Branch,0.0,Pizza Place,Pharmacy,Gym,Skating Rink,Coffee Shop,Pub,Sandwich Place,Dog Run,Diner,Discount Store
96,M9L,Humber Summit,0.0,Pizza Place,Empanada Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
99,M9P,Westmount,0.0,Pizza Place,Intersection,Middle Eastern Restaurant,Discount Store,Chinese Restaurant,Coffee Shop,Sandwich Place,Diner,Distribution Center,Dog Run
100,M9R,Kingsview Village / St. Phillips / Martin Grov...,0.0,Mobile Phone Shop,Pizza Place,Sandwich Place,Bus Line,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Drugstore


#### Cluster 2

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Malvern / Rouge,1.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
2,M1E,Guildwood / Morningside / West Hill,1.0,Moving Target,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Bank,Intersection,Medical Center,Mexican Restaurant,Yoga Studio
4,M1H,Cedarbrae,1.0,Caribbean Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Bank,Hakka Restaurant,Electronics Store,Eastern European Restaurant
7,M1L,Golden Mile / Clairlea / Oakridge,1.0,Bakery,Park,Ice Cream Shop,Bus Line,Metro Station,Intersection,Soccer Field,Coworking Space,Creperie,Falafel Restaurant
10,M1P,Dorset Park / Wexford Heights / Scarborough To...,1.0,Indian Restaurant,Pet Store,Light Rail Station,Vietnamese Restaurant,Chinese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
11,M1R,Wexford / Maryvale,1.0,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
12,M1S,Agincourt,1.0,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Discount Store
13,M1T,Clarks Corners / Tam O'Shanter / Sullivan,1.0,Pharmacy,Pizza Place,Fried Chicken Joint,Gas Station,Fast Food Restaurant,Italian Restaurant,Intersection,Bank,Thai Restaurant,Chinese Restaurant
15,M1W,Steeles West / L'Amoreaux West,1.0,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Thrift / Vintage Store,Nail Salon,Pizza Place,Coffee Shop,Bubble Tea Shop,Breakfast Spot,Supermarket
17,M2H,Hillcrest Village,1.0,Pool,Athletics & Sports,Golf Course,Dog Run,Mediterranean Restaurant,Drugstore,Discount Store,Distribution Center,Doner Restaurant,Donut Shop


#### Cluster 3

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M1M,Cliffside / Cliffcrest / Scarborough Village West,2.0,American Restaurant,Motel,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


#### Cluster 4

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Rouge Hill / Port Union / Highland Creek,3.0,Bar,History Museum,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
3,M1G,Woburn,3.0,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
6,M1K,Kennedy Park / Ionview / East Birchmount Park,3.0,Department Store,Convenience Store,Chinese Restaurant,Hobby Shop,Coffee Shop,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
9,M1N,Birch Cliff / Cliffside West,3.0,College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
18,M2J,Fairview / Henry Farm / Oriole,3.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Bakery,Food Court,Juice Bar,Tea Room,Bank,Chinese Restaurant
19,M2K,Bayview Village,3.0,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
22,M2N,Willowdale,3.0,Coffee Shop,Pizza Place,Ramen Restaurant,Sandwich Place,Restaurant,Café,Discount Store,Sushi Restaurant,Steakhouse,Plaza
24,M2R,Willowdale,3.0,Coffee Shop,Pizza Place,Ramen Restaurant,Sandwich Place,Restaurant,Café,Discount Store,Sushi Restaurant,Steakhouse,Plaza
26,M3B,Don Mills,3.0,Gym,Restaurant,Coffee Shop,Japanese Restaurant,Beer Store,Italian Restaurant,Sporting Goods Shop,Bike Shop,Sandwich Place,Café
27,M3C,Don Mills,3.0,Gym,Restaurant,Coffee Shop,Japanese Restaurant,Beer Store,Italian Restaurant,Sporting Goods Shop,Bike Shop,Sandwich Place,Café


#### Cluster 5

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M1J,Scarborough Village,4.0,Grocery Store,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
14,M1V,Milliken / Agincourt North / Steeles East / L'...,4.0,Park,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
23,M2P,York Mills West,4.0,Park,Bank,Convenience Store,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,M3A,Parkwoods,4.0,Food & Drink Shop,Park,Convenience Store,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
40,M4J,East Toronto,4.0,Intersection,Park,Convenience Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,M4N,Lawrence Park,4.0,Park,Photography Studio,Swim School,Bus Line,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
50,M4W,Rosedale,4.0,Park,Playground,Trail,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
74,M6E,Caledonia-Fairbanks,4.0,Park,Women's Store,Market,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
79,M6L,North Park / Maple Leaf Park / Upwood Park,4.0,Bakery,Park,Construction & Landscaping,Basketball Court,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
90,M8X,The Kingsway / Montgomery Road / Old Mill North,4.0,Park,Smoke Shop,River,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


#### Cluster 6

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[0] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,M9M,Humberlea / Emery,5.0,Baseball Field,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field


#### Cluster 7

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[0] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Postal code,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,M4T,Moore Park / Summerhill East,6.0,Tennis Court,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Fast Food Restaurant


<a id='item5'></a>